# Generate files for training and prediction.
# input is WSGA selected columns

In [4]:
import csv
import pandas as pd
import pysam
import numpy as np
import sys

from matrix_score import matrix_score

In [5]:
FASTA_LOC = '/home/local/ARCS/hq2130/resources/hg19.fasta' # need to modify 

In [6]:
def add_gc_content(info):
    chrom, pos = info['hg19_chr'], float(info['hg19_pos(1-based)'])
    fasta_file = FASTA_LOC
    #fasta_file = '/home/local/ARCS/hq2130/Exome_Seq/resources/hg19.fasta' # on server
    fastafile = pysam.Fastafile(fasta_file)
    seq = fastafile.fetch(chrom, pos, pos + 10).upper()
    gc_count = 0
    for dna in seq:
        if dna in {'G', 'C'}:
            gc_count += 1
    gc_count = gc_count / 10.0
    info['gc_content'] = gc_count
    return info

def add_s_het(info):
    gene = info['genename']
    # s_het
    info['s_het'] = 0
    if gene in s_het:
        info['s_het'] = s_het[gene]
    # s_het log, default for log transform of 0
    info['s_het_log'] = 0 
    if gene in s_het:
        info['s_het_log'] = np.log(s_het[gene]+1)
    # info['s_hat_log'] = -10 # default for log transform of 0
    # if gene in s_hat:
    #     info['s_hat_log'] = np.log(s_hat[gene])
    return info

def add_exac_metric(info):
    info['pli'] = pli.get(info['genename'], '-1')
    info['lofz'] = lofz.get(info['genename'], '0')
    info['prec'] = prec.get(info['genename'], '0')
    return info

def add_target(info, target):
    info['target'] = target_value
    if target_value == 'NA' and 'cancer_target' in info:
        info['target'] = info['cancer_target']  
    elif target_value == 'NA' and 'category' in info:
        if info['category'] == 'TP':
            info['target'] = 1
        elif info['category'] == 'TN':
            info['target'] = 0
    
    return info

def add_gnomad(info):
    info['gnomad'] = 0
    return info
    chrom, pos = info['hg19_chr'], int(info['hg19_pos(1-based)'])
    ref, alt = info['ref'], info['alt']
    if chrom == 'Y':
        return info
    fname = '/home/local/users/jw/resources/gnomad/Genome/gnomad.genomes.r2.0.1.sites.'+chrom+'.vcf.gz'
    tbx = pysam.TabixFile(fname)
    for row in tbx.fetch(chrom, pos-1, pos):
        lst = row.split()
        g_chrom, g_pos, g_ref, g_alt = lst[0], int(lst[1]), lst[3], lst[4]
        if g_chrom == chrom and g_pos == pos and g_ref == ref and g_alt == alt:
            for element in lst[7].split(";"):
                if '=' in element:
                    FieldName,FieldValue=element.split('=',1)
                    if FieldName == 'AF':
                        if FieldValue == '.': FieldValue = 0
                        info['gnomad']=float(FieldValue)
    return info

def add_secondary(info):
    gene, aaref = info['genename'], info['aaref']
    info['secondary_H'] = 0
    info['secondary_C'] = 0
    info['secondary_E'] = 0
    if gene in secondary:
        aapos = info['aapos'].split(';')
        for pos in aapos:
            pos = int(pos)
            # AA_seq start from 0(it's a list)
            protein_length = len(AA_seq[gene])
            if pos < protein_length and AA_seq[gene][pos-1] == aaref:
                if pos in secondary[gene]:
                    if secondary[gene][pos] == 'H':
                        info['secondary_H'] = 1
                    elif secondary[gene][pos] == 'C':
                        info['secondary_C'] = 1    
                    elif secondary[gene][pos] == 'E':
                        info['secondary_E'] = 1
    return info
                                    
def add_BioPlex(info):
    ''' some feather related to protein? added all pint 
        http://bioplex.hms.harvard.edu/downloadInteractions.php
    '''
    gene = info['genename']
    info['BioPlex'] = -1
    if gene in BioPlex:
        info['BioPlex'] = BioPlex[gene]
    return info
        
def choose_HS(info):
    include_variants = False  
    if float(info['pli']) < 0.5 and float(info['pli']) >= 0: # HS genes
        include_variants = True  
    return include_variants

In [7]:
def sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=True):
    """ function that selected colums from wgsa, add some other feathers 
        set . into 0
    """
    with open(fin, 'rU') as f:
        positive, negative = 0, 0
        
        r = csv.reader(f)
        head = r.next()
        feat_all = wgsa_feat + add_feat + extra_feat
        feat = []
        for f in order_feat:
            if f in feat_all:
                feat.append(f)
        for f in feat_all:
            if f not in feat:
                feat.append(f)
        if write_head:
            w.writerow(feat)

        for line in r:
            info = dict(zip(head, line))
            aaref, aaalt, aapos = info['aaref'], info['aaalt'], info['aapos']
            
            var_id = '_'.join([info['hg19_chr'], info['hg19_pos(1-based)'], info['ref'], info['alt']])
            if var_id in exclude_var: continue
            
            # no aaalt 
            if aaref is not "U" and aaalt is not ".":
                # reformat wsga feat, missing value filled with 0, will -1 be better?
                for c in wgsa_feat:
                    if info[c] == '.':
                        info[c] = -1
                    else:
                        info[c] = float(info[c])
                # set some default value for extra_feat
                for c in extra_feat:
                    info[c] = info.get(c, 'NA')
                
                info['genename'] = info['genename']
                info['blosum62'] = matrix_score(aaref, aaalt, 'blosum62')
                info['pam250'] = matrix_score(aaref, aaalt, 'pam250')
                
                
               
                
                # update SUMO/phospho scores
                info['phospho_score'] = 0
                info['phospho_cutoff'] = 0
                info['phospho_diff'] = 0
                gene = info['genename']
                if gene in phosphorylation:
                    aapos = info['aapos'].split(';')
                    for pos in aapos:
                        pos = int(pos)
                        # pos is 1 based
                        if pos in phosphorylation[gene]:
                            if phosphorylation[gene][pos]['AA'] == aaref:
                                info['phospho_score'] = phosphorylation[gene][pos]['Score']
                                info['phospho_cutoff'] = phosphorylation[gene][pos]['Cutoff']
                                info['phospho_diff'] = phosphorylation[gene][pos]['diff']
                                break
                                
                info['SUMO_score'] = 0
                info['SUMO_cutoff'] = 0
                info['SUMO_diff'] = 0
                if gene in SUMO:
                    aapos = info['aapos'].split(';')
                    for pos in aapos:
                        pos = int(pos)
                        # pos is 1 based
                        if pos in SUMO[gene]:
                            if SUMO[gene][pos]['AA'] == aaref:
                                info['SUMO_score'] = SUMO[gene][pos]['Score']
                                info['SUMO_cutoff'] = SUMO[gene][pos]['Cutoff']
                                info['SUMO_diff'] = SUMO[gene][pos]['diff']
                                break  
                                
                # add inteface flag
                info['interface'] = 0
                if gene in interface:
                    aapos = info['aapos'].split(';')
                    for pos in aapos:
                        pos = int(pos)
                        # AA_seq start from 0
                        protein_length = len(AA_seq[gene])
                        if pos < protein_length and AA_seq[gene][pos-1] == aaref:
                            if pos in interface[gene]:
                                info['interface'] = 1
                                
                # add ASA score Accessible Surface Areas
                info['ASA'] = 0
                if gene in ASA:
                    aapos = info['aapos'].split(';')
                    for pos in aapos:
                        pos = int(pos)
                        # AA_seq start from 0
                        protein_length = len(AA_seq[gene])
                        if pos < protein_length and AA_seq[gene][pos-1] == aaref:
                            if pos in ASA[gene]:
                                info['ASA'] = ASA[gene][pos]
                

                                    
                info['ubiquitination'] = 0
                if gene in ubiquitination:
                    aapos = info['aapos'].split(';')
                    for pos in aapos:
                        pos = int(pos)
                        # AA_seq start from 0
                        protein_length = len(AA_seq[gene])
                        if pos < protein_length and AA_seq[gene][pos-1] == aaref:
                            if pos in ubiquitination[gene]:
                                info['ubiquitination'] = ubiquitination[gene][pos]
                
                # gene specific feathers
                info['complex_CORUM'] = 0
                if gene in complex_CORUM:
                    info['complex_CORUM'] = 1
                
                info['preppi_counts'] = 0
                if gene in preppi:
                    info['preppi_counts'] = preppi[gene]
                    
                info = add_secondary(info)
                info = add_exac_metric(info)
                info = add_gc_content(info)
                info = add_s_het(info)
                info = add_BioPlex(info)
                
                info = add_target(info, target_value)
                info = add_gnomad(info)
               
                # choose HIS or HS
                if prefix == '.HS.':
                    include_variants = choose_HS(info)
                else:
                    include_variants = not choose_HS(info)
                    
                include_variants = not choose_HS(info)   
                if include_variants:
                    if info['target'] == 1:
                        positive += 1
                    if info['target'] == 0:
                        negative += 1
                    w.writerow([info[c] for c in feat])
    print '{} pos, {} neg'.format(positive, negative)

In [8]:
# variants excluded for training
exclude_var = set()
with open('../data/excluded_variants_gwas.txt') as f:
    for line in f:
        exclude_var.add(line.strip())  
        
with open('../data/input_data.exclude.txt') as f:
    for line in f:
        exclude_var.add(line.strip())

In [9]:
# Load protein related annotations
SUMO = np.load('../data/protein/SUMO.npy').item()
phosphorylation = np.load('../data/protein/phosphorylation.npy').item()
AA_seq = np.load('../data/protein/AA_seq.npy').item()
interface = np.load('../data/protein/interface.npy').item()
ASA = np.load('../data/protein/ASA.npy').item()
preppi = np.load('../data/protein/preppi.npy').item()
secondary = np.load('../data/protein/secondary.npy').item()
ubiquitination = np.load('../data/protein/ubiquitination.npy').item()
BioPlex = np.load('../data/protein/BioPlex.npy').item()

s_het = np.load('../data/gene/s_het.npy').item()
prec = np.load('../data/gene/prec.npy').item()
pli = np.load('../data/gene/pli.npy').item()
lofz = np.load('../data/gene/lofz.npy').item()


complex_CORUM = set()
with open('../data/protein/protein_complex_CORUM.txt') as f:
    for line in f:
        lst = line.strip().split('\t')
        complex_CORUM = complex_CORUM | set(lst)

In [10]:
# feature order from correlation cluster
order_feat = [u'MutationAssessor_score_rankscore', u'VEST3_rankscore', u'Polyphen2_HDIV_rankscore',
 u'Polyphen2_HVAR_rankscore', u'SIFT_converted_rankscore', u'PROVEAN_converted_rankscore',
 u'MetaSVM_rankscore',u'MetaLR_rankscore', u'FATHMM_converted_rankscore', u'M-CAP_rankscore',
 u'GenoCanyon_score_rankscore', u'LRT_converted_rankscore', u'Eigen-PC-raw_rankscore',
 u'Eigen-phred', u'Eigen-PC-phred', u'DANN_rankscore', u'CADD_phred', u'CADD_raw_rankscore',
 u'phyloP20way_mammalian_rankscore', u'GERP++_RS_rankscore', u'SiPhy_29way_logOdds_rankscore',
 u'phastCons100way_vertebrate_rankscore', u'fathmm-MKL_coding_rankscore', u'phyloP100way_vertebrate_rankscore',
 u'MutationTaster_converted_rankscore', u'phastCons20way_mammalian_rankscore', u'GM12878_fitCons_score_rankscore',
 u'HUVEC_fitCons_score_rankscore', u'integrated_fitCons_score_rankscore',u'H1-hESC_fitCons_score_rankscore', 
 u'blosum62', u'pam250', u'SUMO_diff', u'SUMO_score', u'SUMO_cutoff', u'phospho_cutoff', u'phospho_score',
 u'phospho_diff', u'lofz', u'prec', u'pli',
 u's_het', u's_het_log', u'secondary_E', u'secondary_H', u'complex_CORUM', u'preppi_counts',
 u'1000Gp3_AF', u'ExAC_AF', 'gnomad', u'ASA', u'secondary_C', u'gc_content', u'interface', u'ubiquitination']

In [11]:
# add feathers from WGSA and other inputs, some of them need to be excluded in future training
rank_score_cols = ['SIFT_converted_rankscore', 'Polyphen2_HDIV_rankscore', 'Polyphen2_HVAR_rankscore', 
 'LRT_converted_rankscore', 'MutationTaster_converted_rankscore', 'MutationAssessor_score_rankscore', 
 'FATHMM_converted_rankscore', 'PROVEAN_converted_rankscore', 'VEST3_rankscore', 
 'MetaSVM_rankscore', 'MetaLR_rankscore', 'M-CAP_rankscore', 
 'CADD_raw_rankscore', 'DANN_rankscore', 'fathmm-MKL_coding_rankscore', 
 'Eigen-PC-raw_rankscore', 'GenoCanyon_score_rankscore', 'integrated_fitCons_score_rankscore', 
 'GM12878_fitCons_score_rankscore', 'H1-hESC_fitCons_score_rankscore', 
 'HUVEC_fitCons_score_rankscore', 'GERP++_RS_rankscore', 
 'phyloP100way_vertebrate_rankscore', 'phyloP20way_mammalian_rankscore', 
 'phastCons100way_vertebrate_rankscore', 'phastCons20way_mammalian_rankscore', 
 'SiPhy_29way_logOdds_rankscore']

wgsa_feat = ['1000Gp3_AF', 'ExAC_AF', 'CADD_phred', 'Eigen-phred', 'Eigen-PC-phred']
wgsa_feat = wgsa_feat + rank_score_cols

add_feat =  ['blosum62', 'pam250', 'SUMO_score', 'SUMO_cutoff', 'SUMO_diff',
             'phospho_score', 'phospho_cutoff','phospho_diff', 'interface',
             'ASA', 'pli', 'lofz', 'complex_CORUM', 'preppi_counts',
             'secondary_H', 'secondary_C', 'secondary_E', 'ubiquitination',
             's_het', 'prec', 's_het_log',   'gc_content', 'gnomad', 'BioPlex',
             'target']

# feathers used for future info
extra_feat = ['hg19_chr', 'hg19_pos(1-based)', 
              'ref', 'alt', 'category', 'source','INFO', 'disease', 'genename']

In [12]:
outname = '../data/input_data.csv'
prefix = '.HIS.'

with open(outname, 'w') as fw:
    w = csv.writer(fw)

    # HGMD positive training
    fin = '../data/training/HGMD_DM_missense_anno.rare.csv' 
    target_value = 1
    sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=True)

    # Discover negative data
    fin = '../data/training/DiscovEHR_rare_missense_30000.csv' 
    target_value = 0
    sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=False)
    
    # metaSVM training 
    fin = '../data/training/metaSVM_train.anno.rare.csv' 
    target_value = 'NA'
    sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=False)
    
    # CADD negative data
    fin = '../data/training/CADD_neg_train.anno.rare.csv' 
    target_value = 0
    sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=False)

13716 pos, 0 neg
0 pos, 10216 neg
5918 pos, 2985 neg
0 pos, 12648 neg


In [13]:
# metaSVM testing
fins = ['../data/metaSVM/metaSVM_train.anno.rare.csv', '../data/metaSVM/metaSVM_test1.anno.rare.csv', 
       '../data/metaSVM/metaSVM_test2.anno.rare.csv', '../data/metaSVM/metaSVM_test3.anno.rare.csv', 
       '../data/metaSVM/metaSVM_addtest1.anno.rare.csv', '../data/metaSVM/metaSVM_addtest2.anno.rare.csv',
       '../data/cancer_hotspots/cancer.csv']

fouts = []
for f in fins:
    fouts.append(f.split('.csv')[0] + prefix + 'reformat.csv')
for fin, fout in zip(fins, fouts):
    with open(fout, 'w') as fw:
        w = csv.writer(fw)
        target_value = 'NA'
        sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=True)

5918 pos, 2985 neg
67 pos, 24 neg
2057 pos, 3105 neg
0 pos, 2329 neg
8576 pos, 3389 neg
4846 pos, 2776 neg
0 pos, 0 neg


In [14]:
fins = ['../data/case_control/control_900.anno.rare.csv',
        '../data/case_control/control_1911.anno.rare.csv',
        '../data/case_control/ssc_yale.anno.rare.csv']

fouts = []
for f in fins:
    fouts.append(f.split('.csv')[0] + prefix + 'reformat.csv')

for fin, fout in zip(fins, fouts):
    with open(fout, 'w') as fw:
        w = csv.writer(fw)
        target_value = 0
        sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=True)

0 pos, 249 neg
0 pos, 466 neg
0 pos, 496 neg


In [15]:
fins = ['../data/case_control/case.anno.rare.csv', 
        '../data/case_control/DDD_new_0.2.anno.rare.csv',
        '../data/case_control/chd_yale.anno.rare.csv']

fouts = []
for f in fins:
    fouts.append(f.split('.csv')[0] + prefix +'reformat.csv')

for fin, fout in zip(fins, fouts):
    with open(fout, 'w') as fw:
        w = csv.writer(fw)
        target_value = 1
        sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=True)

3130 pos, 0 neg
2195 pos, 0 neg
893 pos, 0 neg
